## Langchain エージェントと Bedrock モデルの統合

あるアプリケーションでは、ユーザの入力に応じて、言語モデルや様々なユーティリティの呼び出しシーケンスを適応させることが要求されます。エージェントインターフェースは、このようなアプリケーションに柔軟性を提供します。エージェントは様々なリソースを利用することができ、ユーザー入力に基づいて利用するリソースを選択します。エージェントは複数のツールを使用することができ、あるツールの出力を次のツールの入力として利用することができる。

エージェントには、主に2つのカテゴリーがあります。:

- アクションエージェント: 各インターバルで、すべての前のアクションの出力を利用して、次のアクションを決定する。
- 計画実行型エージェント: 最初に行動の完全な順序を決定し、計画を更新することなくすべてを実行する。

このノートブックでは、`plan-and-execute`エージェントと、`Zero-shot ReAct`アクションエージェントを使い、[`ReAct`](https://arxiv.org/pdf/2205.00445.pdf)フレームワークを使って、ツールの説明に基づいて適切なツールを選択するデモを行います。各ツールの説明を提供する必要があります。

## セットアップ

In [ ]:
%pip install google-search-results

In [ ]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['SERPAPI_API_KEY'] = '<SERPAPI_API_KEY>'
boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))

In [ ]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

## ReAct を使う： 言語モデルフレームワークにおける推論と行動の相乗効果
大規模な言語モデルは、推論に対する説明とタスクに特化した応答の両方を交互に生成することができます。

推論説明を生成することで、モデルは行動計画を推論し、監視し、修正し、さらには予期しないシナリオを処理することができます。アクションステップでは、モデルが知識ベースや環境などの外部ソースとインターフェースし、情報を取得することができます。

ReAct フレームワークは、大規模な言語モデルが外部ツールと相互作用することを可能にし、より正確で事実に基づいた回答をもたらす追加情報を得ることができます。

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.utilities import SerpAPIWrapper

In [ ]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
question = "Human: Only answer the question asked. <question> What is Amazon SageMaker? What is its launch year multiplied by 2? </question> Assistant: Launch year multiplied by 2:"


In [ ]:
react_agent.run(question)

## カスタムツール

特定のアクションを実行するために、エージェント内に独自のツールを導入することができます。これらは、データベースのデータを検索したり、API 呼び出しを行ったりします。この例では、エージェントに代わって API 呼び出しを行うツールをスタブアウトします。簡単のために、コードはハードコードされた値を返しますが、独自の統合を作成するために試してみてください。

以下の例では、グーグル検索（SerpAPI）と電卓（LLMMathChain）の2つの組み込みツールを取り込んでいます。また、4つのカスタムツールも追加しています。:
**EC2Search**: タグ名を指定して EC2 インスタンスの検索をシミュレートする。
**EC2Patch**: EC2 インスタンスへのパッチ適用をシミュレートする。
**EC2Stop**: EC2 インスタンスのシャットダウンをシミュレートする。
**ChangeRecord**: CMDB への変更レコードの書き込みをシミュレートします。

この説明文が、どのような場合にそのツールを使うべきかという文脈を提供していることに注目してください。これは重要な要素です。LLM は、どのツール（もしあれば）が質問された問題を最もよく解決するかを判断するために使用します。一語一句一致させる必要はなく、どのツールが最適かをモデル化するために最善を尽くします。

In [ ]:
import json

search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=False)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
tools.append(Tool.from_function(
        name="EC2Search",
        func=lambda x: f"['i-00000000000','i-00000000001','i-00000000002']",  # モック関数、boto3 呼び出しに置き換える
        description="Use this when you need to list EC2 instances in json. It takes a single parameter named tagname"
    ))
tools.append(Tool.from_function(
        name="EC2Patch",
        func=lambda x: f"{len(x.split(','))} instances patched",  # モック関数、boto3 呼び出しに置き換える
        description="Use this when you need to patch EC2 instances"
    ))
tools.append(Tool.from_function(
        name="EC2Stop",
        func=lambda x: f"{len(x.split(','))} instances stopped",  # モック関数、boto3 呼び出しに置き換える
        description="Use this when you need to stop EC2 instances"
    ))

def record_change(x):
    print(f"*{x}*")
    j = json.loads(x[1:-1])
    return f"instance {j['instance']}. ACTION: {j['changeType']} recorded in CMDB" # モック関数、API 呼び出しに置き換える

tools.append(Tool.from_function(
        name="ChangeRecord",
        func=record_change,  # モック関数、API 呼び出しに置き換える
        description="Use this when you need to update a change record in CMDB. This takes in a json document as the parameter. The element named 'instance' contains the instance and the element named 'changeType' is the change type."
    ))

In [ ]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the a tag delete. 
Next, patch each of the instances and record the patch change record in the CMDB with a change type of 'PATCH'. 
Finally, for each of the instances stop the instance and tell me how many were stopped. Assistant:"""

result = react_agent.run(question)

print(f"{result}")

## ツールコードの生成
GenAIを使ってツールのコードを生成できると思いますか？もちろんです！ただし、コードにはばらつきがあります。
人間のレビューなしには実行できないかもしれません。

次の例では、Claude が生成したコードを使用してツールを生成しています。上記の EC2Search ツールを boto3(Python) のコードに置き換えます。

In [ ]:
import xml.etree.ElementTree as ET
from IPython.display import display, Markdown, Latex

prompt_data = """
Human: You are an AI python code generator. You write really great code.

Write a python function named list_tagged_instances with one parameter named tagname. 

The function queries the boto3 library to return a list all of the EC2 instances that have a tag equal to the tagname parameter. 

return the code inside <code></code>.
Computer:"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

tree = ET.ElementTree(ET.fromstring(response_body.get("completion")))
python_code = tree.getroot().text

display(Markdown(f'```{python_code}```'))

exec(python_code)



## コードの実行
上記のコードが妥当であれば、それを使ってエージェントを実行することができる。

**注意 - これを実行するには、このアカウントのEC2インスタンスに「delete」という名前のタグと、そのタグに任意の値を設定する必要があります。大文字小文字を区別するので、これを実行する前にEC2インスタンスを作成し、停止状態に設定しておきましょう。**

In [ ]:
for tool in tools:
    if tool.name == "EC2Search":
        tool.func = list_tagged_instances
        
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the a tag delete. 
Next, patch each of the instances and record the patch change record in the CMDB with a change type of 'PATCH'. 
Finally, for each of the instances stop the instance and tell me how many were stopped. Assistant:"""

result = react_agent.run(question)

print(f"{result}")

## その他の種類 - 存在しないインスタンスを検索する
プロンプトは、存在しないタグを持つEC2インスタンスを検索するように変わるので、インスタンスは返されないはずです。

In [ ]:
question = """Human: Please list my EC2 instances with the a tag doesnotexist. 
Next, patch each of the instances and record the patch change record in the CMDB with a change type of 'PATCH'. 
Finally, for each of the instances stop the instance and tell me how many were stopped. Assistant:"""

result = react_agent.run(question)

print(f"{result}")

## データベースツール
エージェントの一般的な用途は、データベース内のレコードを検索することです。 完全なデータベースをコンテキストに含めることは現実的ではないため、会話的なインタラクションを維持しながら幻覚をなくすような、データベースに対するアクションを実行するツールを提供することができます。

### SQL データベースエージェント
Langachain には、DB に質問して答えを得る方法をデモンストレーションするための SQL Database エージェントがあります。 詳細については、このドキュメントを参照してください: https://python.langchain.com/docs/integrations/toolkits/sql_database

エージェントは DB のスキーマをコンテキストにロードし、自然言語の質問に基づいて SQL ステートメントを生成します。 次に、SQL ステートメントがデータベースに対して実行され、結果が返されます。

### データエージェント
SQL データベースエージェントはデータ探索やクエリの生成に便利ですが、必要な場合もあります。
特定のエンティティについては、データベースからデータを取得してプロンプトに次のステップのコンテキストを提供するツールを作成できます。

以下の例では、customer テーブルの customer に対する DB クエリをシミュレートします。このコードを DynamoDB やリレーションデータベースのルックアップに置き換えてください。

In [ ]:
customer_table=[
  {
    "id": 1, 
    "first_name": "John", 
    "last_name": "Doe",
    "age": 35,
    "postal_code": "90210"
  },
  {  
    "id": 2,
    "first_name": "Jane",
    "last_name": "Smith", 
    "age": 27,
    "postal_code": "12345"
  },
  {
    "id": 3, 
    "first_name": "Bob",
    "last_name": "Jones",
    "age": 42,
    "postal_code": "55555"
  },
  {
    "id": 4,
    "first_name": "Sara", 
    "last_name": "Miller",
    "age": 29, 
    "postal_code": "13579"
  },
  {
    "id": 5,
    "first_name": "Mark",
    "last_name": "Davis",
    "age": 31,
    "postal_code": "02468"
  },
  {
    "id": 6,
    "first_name": "Laura",
    "last_name": "Wilson",
    "age": 24,
    "postal_code": "98765" 
  },
  {
    "id": 7,
    "first_name": "Steve",
    "last_name": "Moore",
    "age": 36,
    "postal_code": "11223"
  },
  {
    "id": 8,
    "first_name": "Michelle",
    "last_name": "Chen",
    "age": 22,
    "orders": [
        {
            "order_id": 1,
            "description": "An order of 1 dozen pencils"
        },
        {
            "order_id": 2,
            "description": "An order of 2 markers"
        }
    ],
    "postal_code": "33215"
  },
  {
    "id": 9,
    "first_name": "David",
    "last_name": "Lee",
    "age": 29,
    "postal_code": "99567"
  },
  {
    "id": 10,
    "first_name": "Jessica",
    "last_name": "Brown",
    "age": 18, 
    "postal_code": "43210"
  }
]

def customer_lookup(id):
    print(f"search by customer {id}")
    for customer in customer_table:
        if customer["id"] == int(id):
            print(f"found customer {id} {customer}")
            return customer
        
    return None

tools.append(Tool.from_function(
        name="CustomerLookup",
        func=customer_lookup,  # Mock Function, replace with an api call
        description="Use this when you need to lookup a customer by id."
    ))

In [ ]:
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: write one sentence summary about the information you know about the customer with an id of 8.

Assistant:"""

result = react_agent.run(question)

print(f"{result}")

### (実験） - プラン・アンド・エグゼキューション・エージェントの使用
計画・実行エージェントは、まず必要なサブタスクとステップを特定することによって行動計画を決定します。そして、目的が達成されるまで、それぞれのサブタスクを順番に実行することで、その計画を実行します。
計画は常に大規模な言語モデルによって行われ、実行は通常、ツールを備えた別のエージェント（実行エージェント）によって行われます。
計画と実行のエージェントは、継続的な集中と協調を必要とする複雑で長期的な目的を扱うのに最も適しています。効果的な戦略は、行動エージェントの応答性と計画・実行エージェントの熟慮的計画能力を統合することです。

In [ ]:

plan_llm = Bedrock(model_id="anthropic.claude-v1", client=boto3_bedrock, model_kwargs=model_parameter)
execute_llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

In [ ]:
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=execute_llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [ ]:
planner = load_chat_planner(plan_llm)
executor = load_agent_executor(execute_llm, tools, verbose=True)
pae_agent = PlanAndExecute(planner=planner, executor=executor, verbose=True, max_iterations=1)

In [ ]:
pae_agent.run("What is Amazon SageMaker? What is its launch year multiplied by 2? Launch year multiplied by 2 is")